In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import base64
#!pip install spotipy
import requests
from bs4 import BeautifulSoup
import time
from spotipy.oauth2 import SpotifyClientCredentials
#!pip install spotipy --upgrade

In [2]:
charts_df = pd.read_csv('../data/charts.csv')
lyrics_df = pd.read_csv('../data/billboard_24years_lyrics_spotify.csv')
spotify_qualities_df = pd.read_csv('../data/Hot100.csv')
song_attributes_df = pd.read_csv('../data/songAttributes_1999-2019.csv')

In [21]:
#spotify_qualities_df.info()
updated_song_attributes_df = song_attributes_df.drop(['Unnamed: 0', 'Popularity', 'Album'], axis=1)
updated_song_attributes_df = updated_song_attributes_df.rename(columns={'Name': 'song', 'Artist': 'artist'})
updated_song_attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154931 entries, 0 to 154930
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Acousticness      154931 non-null  float64
 1   artist            154931 non-null  object 
 2   Danceability      154931 non-null  float64
 3   Duration          154931 non-null  int64  
 4   Energy            154931 non-null  float64
 5   Explicit          154931 non-null  bool   
 6   Instrumentalness  154931 non-null  float64
 7   Liveness          154931 non-null  float64
 8   Loudness          154931 non-null  float64
 9   Mode              154931 non-null  int64  
 10  song              154931 non-null  object 
 11  Speechiness       154931 non-null  float64
 12  Tempo             154931 non-null  float64
 13  TimeSignature     154931 non-null  int64  
 14  Valence           154931 non-null  float64
dtypes: bool(1), float64(9), int64(3), object(2)
memory usage: 16.7+ MB


In [19]:
charts_df['date'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['date'].dt.year
updated_charts_df = charts_df.drop(['date', 'last-week', 'rank'], axis=1)
updated_charts_df.head(10)

,song,artist,peak-rank,weeks-on-board,year,datetime
0,Easy On Me,Adele,1,3,2021,2021-11-06
1,Stay,The Kid LAROI & Justin Bieber,1,16,2021,2021-11-06
2,Industry Baby,Lil Nas X & Jack Harlow,1,14,2021,2021-11-06
3,Fancy Like,Walker Hayes,3,19,2021,2021-11-06
4,Bad Habits,Ed Sheeran,2,18,2021,2021-11-06
5,Way 2 Sexy,Drake Featuring Future & Young Thug,1,8,2021,2021-11-06
6,Shivers,Ed Sheeran,7,7,2021,2021-11-06
7,Good 4 U,Olivia Rodrigo,1,24,2021,2021-11-06
8,Need To Know,Doja Cat,9,20,2021,2021-11-06
9,Levitating,Dua Lipa,2,56,2021,2021-11-06


In [33]:
chart_and_att_df = pd.merge(updated_charts_df, updated_song_attributes_df, on=['song', 'artist'], how='inner')
grouped_chart_and_att_df = chart_and_att_df.groupby('song').first().reset_index()
grouped_chart_and_att_df.head(20)
grouped_chart_and_att_df['artist'].value_counts()

artist
Taylor Swift            71
Drake                   69
Tim McGraw              42
Stevie Wonder           39
George Strait           34
                        ..
Ben Folds                1
Lostprophets             1
Lloyd                    1
Down A.K.A. Kilo         1
A Rocket To The Moon     1
Name: count, Length: 736, dtype: int64

In [34]:
grouped_lyrics_df = lyrics_df.groupby('song').first().reset_index()
grouped_lyrics_df

,song,rank,band_singer,songurl,titletext,url,year,lyrics,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68
0,I dare to affirm,Then,notwithstanding all I have said before,your fathers stooped,basely stoopedTo palter with us in a double s...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,I have been often aroused by the dead heavy f...,In the deep still darkness of midnight,and the piteous cries of the chained gangs th...,when speaking to my mistress in the morning,to hear her say that the custom was very wick...,and the heart-rending cries. I was glad to fi...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,I would say,For my part,welcome infidelity! welcome atheism! welcome ...,as preached by those Divines! They convert th...,and barbarous cruelty,and serve to confirm more infidels,in this age,than all the infidel writings of Thomas Paine,Voltaire,and Bolingbroke,put together,have done! These ministers make religion a co...,having neither principles of right action,nor bowels of compassion. They strip the love...,and leave the throng of religion a huge,horrible,repulsive form. It is a religion for oppressors,tyrants,man-stealers,and thugs. It is not that “pure and undefiled...,and which is “first pure,then peaceable,easy to be entreated,full of mercy and good fruits,without partiality,and without hypocrisy.” But a religion which ...,tyrants and slaves; which says to the man in ...,stay there; and to the oppressor,oppress on; it is a religion which may be pro...,denies his fatherhood of the race,and tramples in the dust the great truth of t...,and the popular worship of our land and natio...,a church,and a worship which,on the authority of inspired wisdom,we pronounce to be an abomination in the sigh...,the American church might be well addressed,“Bring no more vain ablations; incense is an ...,the calling of assemblies,I cannot away with; it is iniquity even the s...,I will not hear. YOUR HANDS ARE FULL OF BLOOD...,learn to do well; seek judgment; relieve the ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,and,I take this law to be one of the grossest infr...,if the churches and ministers of our country ...,or most wickedly indifferent,they,too,would so regard it.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,and drive them,The flesh-mongers gather up their victims by d...,chained,to the general depot at Baltimore. When a suf...,a ship is chartered,for the purpose of conveying the forlorn crew...,or to New Orleans. From t

In [6]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            330087 non-null  datetime64[ns]
 1   rank            330087 non-null  int64         
 2   song            330087 non-null  object        
 3   artist          330087 non-null  object        
 4   last-week       297775 non-null  float64       
 5   peak-rank       330087 non-null  int64         
 6   weeks-on-board  330087 non-null  int64         
 7   year            330087 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(2)
memory usage: 18.9+ MB


In [7]:
charts_df['artist'].value_counts()

artist
Taylor Swift                                      1023
Elton John                                         889
Madonna                                            857
Drake                                              787
Kenny Chesney                                      769
                                                  ... 
RiceGum Featuring Alissa Violet                      1
XXXTENTACION Featuring PnB Rock & Trippie Redd       1
Justin Bieber Featuring Burna Boy                    1
Justin Bieber Featuring BEAM                         1
Frankie Vaughan                                      1
Name: count, Length: 10205, dtype: int64

In [8]:
lyrics_df = lyrics_df.rename(columns={'ranking': 'rank'})
spotify_qualities_df = spotify_qualities_df.rename(columns={'Time_Signature': 'time_signature', 'Track': 'song', 'Year': 'year', 'Danceability': 'danceability', 'Energy': 'energy', 'Key': 'key', 'Loudness': 'loudness', 'Mode': 'mode', 'Speechiness': 'speechiness', 'Liveness': 'liveness', 'Valence': 'valence', 'Tempo': 'tempo'}) 
spotify_qualities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song              620 non-null    object 
 1   Artist            620 non-null    object 
 2   Album             620 non-null    object 
 3   year              620 non-null    int64  
 4   Duration          620 non-null    int64  
 5   time_signature    620 non-null    int64  
 6   danceability      620 non-null    float64
 7   energy            620 non-null    float64
 8   key               620 non-null    int64  
 9   loudness          620 non-null    float64
 10  mode              620 non-null    int64  
 11  speechiness       620 non-null    float64
 12  Acousticness      620 non-null    float64
 13  Instrumentalness  620 non-null    float64
 14  liveness          620 non-null    float64
 15  valence           620 non-null    float64
 16  tempo             620 non-null    float64
 1

In [9]:
charts_df['datetime'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['datetime'].dt.year

In [10]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            330087 non-null  datetime64[ns]
 1   rank            330087 non-null  int64         
 2   song            330087 non-null  object        
 3   artist          330087 non-null  object        
 4   last-week       297775 non-null  float64       
 5   peak-rank       330087 non-null  int64         
 6   weeks-on-board  330087 non-null  int64         
 7   year            330087 non-null  int32         
 8   datetime        330087 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int32(1), int64(3), object(2)
memory usage: 21.4+ MB


In [11]:
charts_and_lyrics_df = pd.merge(charts_df, lyrics_df, on=['song'], how='inner')
pd.set_option('display.max_columns', None)
charts_and_lyrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100092 entries, 0 to 100091
Data columns (total 77 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   date              100092 non-null  datetime64[ns]
 1   rank_x            100092 non-null  int64         
 2   song              100092 non-null  object        
 3   artist            100092 non-null  object        
 4   last-week         94523 non-null   float64       
 5   peak-rank         100092 non-null  int64         
 6   weeks-on-board    100092 non-null  int64         
 7   year_x            100092 non-null  int32         
 8   datetime          100092 non-null  datetime64[ns]
 9   rank_y            100092 non-null  object        
 10  band_singer       100092 non-null  object        
 11  songurl           100092 non-null  object        
 12  titletext         100092 non-null  object        
 13  url               100092 non-null  object        
 14  year

In [12]:
range_charts_and_lyrics_df = charts_and_lyrics_df[
    (charts_and_lyrics_df['year'] >= 1999) &
    (charts_and_lyrics_df['year'] <= 2023)]
no_repeats_range_charts_and_lyrics_df = range_charts_and_lyrics_df.groupby('song').first().reset_index()
no_repeats_range_charts_and_lyrics_df.head(10)

KeyError: 'year'

In [ ]:
#charts_and_lyrics_qualities_df = pd.merge(charts_and_lyrics_df, spotify_qualities_df, on=['year', 'song', 'time_signature', 'danceability', 'energy', 'loudness', 'speechiness', 'liveness', 'valence', 'tempo'], how='inner')
#pd.set_option('display.max_columns', None)
#charts_and_lyrics_qualities_df= charts_and_lyrics_qualities_df['song'].nunique()
#charts_and_lyrics_qualities_df.head(30)

In [ ]:
artists = sorted(charts_and_lyrics_df['artist'].astype(str).unique().tolist())
#artists

In [ ]:
#New columns for Main and Featured Artists
featured_artists= ['Featuring', '&', 'With']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

charts_and_lyrics_df['featured_artist'] = charts_and_lyrics_df['artist'].apply(extract_featured)
charts_and_lyrics_df['main_artist'] = charts_and_lyrics_df['artist'].apply(extract_main_artist)
charts_and_lyrics_df['main_artist'].head(20)

In [ ]:
spotify_qualities_df.dtypes

In [ ]:
#lyrics_df['time_signature'].value_counts()
spotify_qualities_df['time_signature'].value_counts()

In [ ]:
charts_and_lyrics_df[charts_and_lyrics_df['time_signature'] == 5]

In [ ]:
spotify_qualities_df['year'].max()
charts_and_lyrics_df['year'].max()

In [ ]:
charts_and_lyrics_df.info()

In [ ]:
updated_charts_and_lyrics_df = no_repeats_range_charts_and_lyrics_df.combine_first(spotify_qualities_df.set_index('song')).reset_index()
updated_charts_and_lyrics_df.head()